# Faultproof Withdrawals Triage Runbook
In order to start this runbook locally please use the README located at [https://github.com/ethereum-optimism/monitorism/blob/main/op-monitorism/faultproof_withdrawals/runbooks/automated/README.md](https://github.com/ethereum-optimism/monitorism/blob/main/op-monitorism/faultproof_withdrawals/runbooks/automated/README.md)

General instructions to start the runbook locally:
```bash
cd op-monitorism/faultproof_withdrawals/runbooks/automated/
make start
```


In [ ]:
from dotenv import load_dotenv
import os
from web3 import Web3
from lib.superchain import *
from lib.web3 import *
from pprint import pprint

#parameters setup (there should be no need to change the one below)
abi_folder_path="abi"

env_file = ".env"
if os.path.exists(env_file):
    load_dotenv(env_file)
    print("Environment variables loaded from .env file.")
else:
    print("No .env file found. Using system environment variables. Make sure to set them up. An example .env file is provided insite the automated folder .env.example")

L1_GETH_URL = os.getenv('L1_GETH_URL')
L2_OP_NODE_URL = os.getenv('L2_OP_NODE_URL')
L2_OP_GETH_URL = os.getenv('L2_OP_GETH_URL')
L2ToL1MessagePasserAddress="0x4200000000000000000000000000000000000016"

## setup 
In order to proceed make sure you set the parameters below.
The url used below are the trusted nodes that are going to be used.

In [27]:
#set ignore_certificate if you are using a local https
ignore_url_certificate=True

#set cahin you need to monitor
l1_chain_name="mainnet"
l2_chain_name="op"

Txhash=""
#Txhash="0xf290e62898913197bc96033a29c098aeb26613d504cfa51bf5ca5ac90cca58da"# Example of a Txhash


## Loading local values and superchain regsitry values

In [ ]:
superchain=get_superchain_file(l1_chain_name, l2_chain_name)

eth_scan_url="https://etherscan.io"

if l1_chain_name=="sepolia":
    eth_scan_url="https://sepolia.etherscan.io"

l2_eth_scan_url=superchain["explorer"]

OptimismPortalProxy=superchain["addresses"]["OptimismPortalProxy"]

print(f"OptimismPortal2 address: {eth_scan_url}/address/{OptimismPortalProxy}#readProxyContract")

web3_utility=Web3Utility(L1_GETH_URL, L2_OP_GETH_URL,L2_OP_NODE_URL,abi_folder_path, OptimismPortalProxy, ignore_certificate=ignore_url_certificate)


# Triage alerts: 
The following actions are useful for triaging the following alerts:
1. faultproof-withdrawal-forgery-detected
2. faultproof-potential-withdrawal-forgery-detected
3. faultproof-suspicious-withdrawal-forgery-detected

An event is considered a forgery if any of the following conditions apply:
1. The withdrawalHash is not present on L2. We check this by querying [L2ToL1MessagePasser](https://github.com/ethereum-optimism/optimism/blob/develop/packages/contracts-bedrock/src/L2/L2CrossDomainMessenger.sol).
2. The [outputRoot provided](https://github.com/ethereum-optimism/optimism/blob/dd2b21ce786f4c1b722bda270348597182153c8e/packages/contracts-bedrock/src/L1/OptimismPortal2.sol#L314C15-L314C25) does not match what we see on [L2 block rootState](https://github.com/ethereum-optimism/monitorism/blob/c0b2ecdf4404888e5ceccf6ad14e35c5e5c52664/op-monitorism/faultproof_withdrawals/validator/op_node_helper.go#L47).

During the triage we need to make sure that the found withdrawalHash is really a malicious one.

Given a withdrawal event hash, that refers to this alert we are going to look for the corresponding game and verify the withdrawal is truly malicious.

In [ ]:

if Txhash=="":
    print("Please set up a Txhash")
else:
    res=web3_utility.get_withdrawal_proven_extension_1(Txhash)
    res=res[0]
    transactionHash=res["transactionHash"].hex()
    print(f"transactionHash: {eth_scan_url}/tx/0x{transactionHash}")
    proofSubmitter=res["args"]["proofSubmitter"]
    print(f"proofSubmitter: {eth_scan_url}/address/{proofSubmitter}")
    withDrawalHash=res["args"]["withdrawalHash"].hex()
    print(f"withdrawalHash: 0x{withDrawalHash}")

    gameData=web3_utility.get_game_data(withDrawalHash,proofSubmitter)
    gameProxyAddress=gameData["gameProxyAddress"]
    print(f"gameProxyAddress: {eth_scan_url}/address/{gameProxyAddress}")

    if gameData["rootClaim"] == gameData["optimism_outputAtBlock"]:
        print(f"rootClaim: {gameData['rootClaim']} == optimism_outputAtBlock: {gameData['optimism_outputAtBlock']}")
        print("\nALL GOOD: Game claim is valid")
    else:
        print(f"rootClaim: {gameData['rootClaim']} != optimism_outputAtBlock: {gameData['optimism_outputAtBlock']}")
        print("WARNING: game claim is invalid")

    if gameData["sentMessages"] == True:
        print("sentMessages: True")
        print("\nALL GOOD Withdrawal present on L2.")
    else:
        print("sentMessages: False")
        print("\nWARNING: Withdrawal not present on L2.")
    print(f"Can be verified by checking {l2_eth_scan_url}/address/{L2ToL1MessagePasserAddress}#readProxyContract and calling sentMessages function with the withdrawalHash 0x{withDrawalHash}")

